# Tools with MCP ⏰

The Model Context Protocol (MCP) provides a standardized way to connect AI agents to external tools and data sources. Let's connect to an MCP server using `langchain-mcp`.

In [7]:
from langchain_mcp_adapters.client import MultiServerMCPClient
import nest_asyncio

nest_asyncio.apply()

# Connect to the mcp-time server for timezone-aware operations
# This Go-based server provides tools for current time, relative time parsing,
# timezone conversion, duration arithmetic, and time comparison
mcp_client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "npx",
            "args": ["-y", "@theo.foobar/mcp-time"],
        }
    },
)

# Load tools from the MCP server
mcp_tools = await mcp_client.get_tools()
print(f"Loaded {len(mcp_tools)} MCP tools: {[t.name for t in mcp_tools]}")

Loaded 5 MCP tools: ['add_time', 'compare_time', 'convert_timezone', 'current_time', 'relative_time']


Create an agent with the MCP-provided time tools.

In [8]:
from langchain.agents import create_agent

agent_with_mcp = create_agent(
    model="openai:gpt-5-mini",
    tools=mcp_tools,
    system_prompt="You are a helpful assistant",
)

Ask about the current time in San Francisco.

In [10]:
result = await agent_with_mcp.ainvoke(
    {"messages": [{"role": "user", "content": "What's the current time in San Francisco right now?"}]}
)
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What's the current time in San Francisco right now?
================================== Ai Message ==================================
Tool Calls:
  current_time (call_B33ZK7m1NoMprhwbYTdtLjV5)
 Call ID: call_B33ZK7m1NoMprhwbYTdtLjV5
  Args:
    timezone: America/Los_Angeles
================================= Tool Message =================================
Name: current_time

2025-10-08T10:40:47-07:00
================================== Ai Message ==================================

It's 10:40:47 AM PDT (UTC−07:00) on 2025-10-08 in San Francisco.
